In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
from glob import glob

import pandas as pd
import pycountry
from bokeh.io import output_notebook, show
from bokeh.models import (
    ColumnDataSource,
    FactorRange,
    HoverTool,
    LabelSet,
    Legend,
    Title,
)
from bokeh.palettes import Category20
from bokeh.plotting import figure

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
raw_data_files = "app/data/raw/data2_*.csv"
processed_data_file_path = "app/data/raw/data.parquet.gzip"
new_col_names = ["Year", "Country", "Value"]
d = {
    "Bolivia (Plurinational State of)": "BOL",
    "Democratic People's Republic of Korea": "PRK",
    "Democratic Republic of the Congo": "COD",
    "Iran (Islamic Republic of)": "IRN",
    "Micronesia (Federated States of)": "FSM",
    "Republic of Korea": "KOR",
    "Republic of Moldova": "MDA",
    "United Kingdom of Great Britain and Northern Ireland": "GBR",
    "United Republic of Tanzania": "TZA",
    "United States of America": "USA",
    "Venezuela (Bolivarian Republic of)": "VEN",
}

In [5]:
files = glob(raw_data_files)

In [6]:
dfs = [pd.read_csv(file, header=1) for file in files]
dfs = []
max_countries_idx = 0
max_countries = 0
for f_idx, file in enumerate(files):
    df = pd.read_csv(file, header=1)
    df = df[df["Beverage Types"] == " All types"].drop(
        columns=["Beverage Types", "Data Source"], axis=1
    )
    if df["Country"].nunique() > max_countries:
        max_countries_idx = f_idx
        max_countries = df["Country"].nunique()
    dfs.append(df)
    # print(df["Country"].nunique(), file, f_idx, max_countries)

nmdf = [dfs[i] for i, v in enumerate(dfs) if i != max_countries_idx]
df = dfs[max_countries_idx]
for df_i in nmdf:
    df = df.merge(df_i, how="left", on=["Country"])
    print(df.shape)
display(df.head())

(188, 20)
(189, 40)
(189, 60)


,Country,2018,2017,2016,2015,2014,2013,2012,2011,2010,...,1969,1968,1967,1966,1965,1964,1963,1962,1961,1960
0,Afghanistan,0.01,0.01,0.02,0.00,0.01,0.01,0.01,0.01,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN
1,Albania,4.70,4.75,4.67,4.54,4.33,4.54,4.74,5.42,5.28,...,1.73,1.44,1.17,1.03,1.04,1.04,0.91,1.17,NaN,NaN
2,Algeria,0.58,0.58,0.60,0.58,0.54,0.50,0.43,0.40,0.35,...,1.87,1.32,0.88,0.85,0.93,0.91,1.12,1.77,2.57,NaN
3,Andorra,9.75,9.93,9.94,9.85,9.89,10.07,10.34,10.46,10.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,5.35,5.47,5.38,5.94,7.49,7.01,7.14,6.95,6.83,...,5.51,5.39,5.25,5.76,5.31,4.92,4.70,4.49,5.37,NaN


Inspect and drop duplicates

In [7]:
df_dups = df.loc[df.duplicated(subset=["Country"], keep=False)]
display(df_dups)

,Country,2018,2017,2016,2015,2014,2013,2012,2011,2010,...,1969,1968,1967,1966,1965,1964,1963,1962,1961,1960
116,Nauru,2.17,2.17,2.17,2.17,2.17,2.17,3.2,3.98,4.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,Nauru,2.17,2.17,2.17,2.17,2.17,2.17,3.2,3.98,4.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Reshape

In [8]:
df = df.set_index(["Country"]).unstack().reset_index()
df.columns = new_col_names
df = df.dropna()

Add country codes

In [9]:
codes = []
for index, row in df.iterrows():
    country = row["Country"].split(r" (")[0]
    try:
        code = pycountry.countries.get(name=country).alpha_3
    except:
        if row["Country"] in list(d.keys()):
            code = d[row["Country"]]
        else:
            code = row["Country"]
    codes.append(code)
df["Code"] = codes
display(df.head())
display(df.tail())

,Year,Country,Value,Code
0,2018,Afghanistan,0.01,AFG
1,2018,Albania,4.70,ALB
2,2018,Algeria,0.58,DZA
3,2018,Andorra,9.75,AND
4,2018,Angola,5.35,AGO


,Year,Country,Value,Code
10959,1961,Yemen,0.70,YEM
10960,1961,Zambia,8.46,ZMB
10961,1961,Zimbabwe,4.24,ZWE
10971,1960,Austria,8.91,AUT
11142,1960,United States of America,7.83,USA


Wrapping this code in a helper function

In [10]:
# df = combine_process_data(files, new_col_names, d)
# display(df.head())
# display(df.tail())
# display(df.dtypes.to_frame())

Export processed data to disk

In [11]:
# df.to_parquet(
#     processed_data_file_path, compression="gzip", index=False, engine="pyarrow"
# )

Example of loading processed data while filtering rows

In [12]:
def bokeh_create_multiline_plot(
    data,
    x,
    x_start,
    x_end,
    y_names,
    lw=2,
    ptitle="Title",
    t_str="",
    t_loc="above",
    axis_tick_font_size="12pt",
    plot_titleFontSize="14pt",
    legend_axis_gap=5,
    fig_size=(700, 400),
):
    col_names = data.column_names[1:]
    color = Category20[len(data.column_names[1:])]
    p = figure(
        plot_width=fig_size[0], plot_height=fig_size[1], toolbar_location=None, tools=""
    )
    p_dict = dict()
    for col, c, col_name in zip(y_names, color, col_names):
        p_dict[col_name] = p.line(
            x,
            col,
            source=data,
            color=c,
            line_width=lw,
            line_alpha=1.0,
            line_color=c,
        )
        p.add_tools(
            HoverTool(
                renderers=[p_dict[col_name]],
                # show_arrow=False,
                # line_policy="next",
                tooltips=[(x, f"@{x}"), (col, f"@{col}")],
            )
        )

    legend = Legend(
        items=[(x, [p_dict[x]]) for x in p_dict],
        location="top_right",
        orientation="vertical",
    )
    p.hover.point_policy = "follow_mouse"
    p.add_layout(legend, "right")
    p.legend.border_line_alpha = 0
    p.legend.background_fill_alpha = 0
    p.xaxis.major_label_text_font_size = axis_tick_font_size
    p.yaxis.major_label_text_font_size = axis_tick_font_size
    p.title.text_font_size = plot_titleFontSize
    p.legend.label_text_font_size = axis_tick_font_size
    p.title.text = f"{ptitle}, {x_start}-{x_end}"
    p.legend.border_line_width = 0
    p.legend.padding = 0
    p.legend.margin = legend_axis_gap
    show(p)

In [13]:
def get_line_chart_data(
    processed_data_file_path,
    nlargest=20,
    year_start=1961,
    year_end=2010,
    groupby_col="Country",
    x="Year",
    z="Value",
):
    df = pd.read_parquet(
        processed_data_file_path,
        filters=[("Year", ">=", year_start), ("Year", "<=", year_end)],
        engine="pyarrow",
    )
    countries = df.groupby([groupby_col])[z].mean().nlargest(nlargest).index.tolist()
    df = df.loc[df[groupby_col].isin(countries)]
    df = df.pivot(index=x, columns=[groupby_col], values=z)
    df = df.loc[:, :].div(df.iloc[1, :])
    return df

In [14]:
x_start = 1960
x_end = 2018
source = ColumnDataSource(data=dict(Year=[], Austria=[]))
df = get_line_chart_data(
    processed_data_file_path,
    15,
    x_start,
    x_end,
    groupby_col="Code",
    x="Year",
    z="Value",
)
source.data = df
bokeh_create_multiline_plot(
    source,
    "Year",
    df.index.min(),
    df.index.max(),
    df.columns,
    2,
    ptitle="Relative Alcohol Consumption",
    axis_tick_font_size="12pt",
    plot_titleFontSize="14pt",
    legend_axis_gap=5,
    fig_size=(700, 400),
)